In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
from scipy import signal as sg
from scipy import integrate
from scipy.interpolate import interpn,interp1d,RegularGridInterpolator, Akima1DInterpolator

import time
from tqdm import notebook
import importlib

import torch
from torch.utils.data import TensorDataset, DataLoader

import tarfile

%aimport -sg -mpl -plt -notebook -np -integrate -torch -TensorDataset -DataLoader

%matplotlib inline

In [2]:
import sys
from importlib import reload,import_module
import os

import sagemaker
import boto3

In [7]:
%env AWS_DEFAULT_REGION=eu-west-2
%cd ..

env: AWS_DEFAULT_REGION=eu-west-2
/mnt/d/Google Drive/projects/paris-traffic-forecast


In [4]:
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix_in  = 'paris-traffic-forecast/input'
prefix_out = 'paris-traffic-forecast/model'
role = "SageMakerRole"

In [8]:
rep_data = 'tmp'

input_channel = sagemaker_session.upload_data(path=rep_data, bucket=bucket, key_prefix=prefix_in)
print('input file (in this case, just an S3 path): {}'.format(input_channel))


input file (in this case, just an S3 path): s3://sagemaker-eu-west-2-047892910832/paris-traffic-forecast/input


2. Creation des instances et paramétrage de l'estimateur:

In [9]:
output_path = f's3://'+bucket+'/'+prefix_out
output_path

's3://sagemaker-eu-west-2-047892910832/paris-traffic-forecast/model'

In [10]:
from sagemaker.pytorch import PyTorch
framework_version = '1.9'

In [13]:
max_run = 60*60*6 # Max time in seconds

estimator = PyTorch(entry_point = 'train_TFT.py',
                                role = role,
                                source_dir  = 'src',
                                py_version = 'py38',
                                max_run=max_run,
                                framework_version = framework_version,
                                instance_count = 1,
                                instance_type='ml.p3.2xlarge', #'ml.m4.xlarge',#'ml.p2.xlarge',#'ml.p3.2xlarge',#
                                output_path=f's3://'+bucket+'/'+prefix_out,
                                hyperparameters={
                                                    'train_fileName':'voi-convention2021.pkl',
                                                    'pretrain_epochs':20,
                                                    'finetune_epochs':10,

                                }
                               )
input_channel

's3://sagemaker-eu-west-2-047892910832/paris-traffic-forecast/input'

In [14]:
estimator.fit({'training': input_channel})

2021-12-12 04:10:32 Starting - Starting the training job...
2021-12-12 04:11:01 Starting - Launching requested ML instancesProfilerReport-1639282232: InProgress
...
2021-12-12 04:11:25 Starting - Insufficient capacity error from EC2 while launching instances, retrying!...
2021-12-12 04:12:03 Failed - Training job failed
ProfilerReport-1639282232: Stopping
.

In [ ]:
training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
display('Model is here: {}'.format(trained_model_location))

s3 = boto3.resource('s3')    
s3.Bucket(bucket).download_file(prefix_out + '/' + desc['TrainingJobName'] + '/' + 'output/model.tar.gz','./models/model.tar.gz')

In [42]:
model_dir = './models/model0'

with tarfile.open('./models/model.tar.gz','r:gz') as archived:
    archived.extractall(model_dir)